# Proyecto de Películas con BeautifulSoup

## Parte 1). Importación de librerías. Ingesta de cada dataset, lectura, creación de una nueva columna y concatenación.

In [1]:
import pandas as pd
import numpy as np
import re
import requests                 #para obtener una respuesta de una pagina web
import bs4                      # para trabajar Web scrapin
from bs4 import BeautifulSoup
import random 
import time 

#### Leemos primero los 4 data set que nos han dado

In [6]:
path = "/home/eloy/Documentos/Mi_primer_proyecto/Datasets/"

In [7]:
df_amazon = pd.read_csv(path + "amazon_prime_titles.csv", sep = ",")
df_disney = pd.read_csv(path + "disney_plus_titles.csv", sep = ",")
df_hulu = pd.read_csv(path + "hulu_titles.csv", sep = ",")
df_netflix = pd.read_json(path + "netflix_titles.json")

#### Creamos una vista de cada uno de los dataframe

In [7]:
df_amazon.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


In [8]:
df_disney.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.
3,s4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,"November 26, 2021",2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!"
4,s5,TV Show,The Beatles: Get Back,NaN,"John Lennon, Paul McCartney, George Harrison, ...",NaN,"November 25, 2021",2021,NaN,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...


In [9]:
df_hulu.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,"October 24, 2021",2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...
1,s2,Movie,Silent Night,NaN,NaN,NaN,"October 23, 2021",2020,NaN,94 min,"Crime, Drama, Thriller","Mark, a low end South London hitman recently r..."
2,s3,Movie,The Marksman,NaN,NaN,NaN,"October 23, 2021",2021,PG-13,108 min,"Action, Thriller",A hardened Arizona rancher tries to protect an...
3,s4,Movie,Gaia,NaN,NaN,NaN,"October 22, 2021",2021,R,97 min,Horror,A forest ranger and two survivalists with a cu...
4,s5,Movie,Settlers,NaN,NaN,NaN,"October 22, 2021",2021,NaN,104 min,"Science Fiction, Thriller",Mankind's earliest settlers on the Martian fro...


In [10]:
df_netflix.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",None,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,None,None,None,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,None,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


#### Crearemos una columna llamada "plataforma" en cada uno de los dataset con su repectivo nombre

In [11]:
# para el dataset de amazon
df_amazon["plataforma"] = "Amazon premier"

# para el dataset de disney
df_disney["plataforma"] = "Disney plus"

# para el dataset de hulu
df_hulu["plataforma"] = "Hulu"

# para el dataset de amazon
df_netflix["plataforma"] = "Netflix"

### Concatenamos los 4 data set

In [12]:
# Concatenamos
df_concat = pd.concat([df_amazon, df_disney, df_hulu, df_netflix], ignore_index = True)

In [13]:
# crearemos una vista
df_concat

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...,Amazon premier
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,Amazon premier
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,Amazon premier
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ...",Amazon premier
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,Amazon premier
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22993,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a...",Netflix
22994,s8804,TV Show,Zombie Dumb,None,None,None,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g...",Netflix
22995,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...,Netflix
22996,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero...",Netflix


## Parte 2). Limpieza del dataframe df_concat

In [ ]:
# Mostraemos la información del dataframe
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       22998 non-null  object
 1   type          22998 non-null  object
 2   title         22998 non-null  object
 3   director      14739 non-null  object
 4   cast          17677 non-null  object
 5   country       11499 non-null  object
 6   date_added    13444 non-null  object
 7   release_year  22998 non-null  int64 
 8   rating        22134 non-null  object
 9   duration      22516 non-null  object
 10  listed_in     22998 non-null  object
 11  description   22994 non-null  object
 12  plataforma    22998 non-null  object
dtypes: int64(1), object(12)
memory usage: 2.3+ MB


In [ ]:
# Veamos si hay duplicados
df_concat[df_concat.duplicated()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma


In [15]:
# Veamos cuantos valores nulos tiene cada columna
df_concat.isna().sum()

show_id             0
type                0
title               0
director         8259
cast             5321
country         11499
date_added       9554
release_year        0
rating            864
duration          482
listed_in           0
description         4
plataforma          0
dtype: int64

#### Comenzaremos hacer el análisis de atras para adelante, esto es, veremos solamente aquellas columnas, la cual nos interesan para la API, que contienen muy poco valores nulos

In [ ]:
df_concat[df_concat["duration"].isnull()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma
11118,s1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,"October 24, 2021",2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...,Hulu
11130,s13,Movie,Out of the Shadows: The Man Behind the Steele ...,NaN,NaN,NaN,"October 18, 2021",2021,NaN,NaN,News,George Stephanopoulos sits down with former MI...,Hulu
11133,s16,Movie,Showtime Championship Boxing: Lopez vs. Salido...,NaN,NaN,NaN,"October 16, 2021",2011,NaN,NaN,Sports,Juan Manuel Lopez vs. Orlando Salido for the W...,Hulu
11155,s38,Movie,BELLATOR MMA: Kongo vs. Johnson 2,NaN,NaN,NaN,"October 13, 2021",2020,NaN,NaN,Sports,Cheick Kongo vs. Tim Johnson battle in a histo...,Hulu
11158,s41,Movie,In the Crease,NaN,NaN,NaN,"October 13, 2021",2020,NaN,NaN,Sports,Post-game analysis and highlight show airing e...,Hulu
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14077,s2960,Movie,Packed to the Rafters,NaN,NaN,Australia,"December 3, 2013",2008,6 Seasons,NaN,"Comedy, Drama, International",Packed to the Rafters is an Australian family-...,Hulu
14111,s2994,Movie,Amy Winehouse: A Final Goodbye,NaN,NaN,United Kingdom,"December 13, 2012",2011,71 min,NaN,"Documentaries, Music",A Final Goodbye is an intimate look into the l...,Hulu
19732,s5542,Movie,Louis C.K. 2017,Louis C.K.,Louis C.K.,United States,"April 4, 2017",2017,74 min,None,Movies,"Louis C.K. muses on religion, eternal love, gi...",Netflix
19985,s5795,Movie,Louis C.K.: Hilarious,Louis C.K.,Louis C.K.,United States,"September 16, 2016",2010,84 min,None,Movies,Emmy-winning comedy writer Louis C.K. brings h...,Netflix


#### Notese en lo anterior que la columna "rating" tiene algunos valores que pertecen a la columna "duration"

In [ ]:
df_concat["rating"].unique()

array([nan, '13+', 'ALL', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+',
       'TV-PG', '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'G',
       'PG', 'NC-17', 'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES',
       'NOT_RATE', 'TV-Y7-FV', 'NOT RATED', '2 Seasons', '93 min',
       '4 Seasons', '136 min', '91 min', '85 min', '98 min', '89 min',
       '94 min', '86 min', '3 Seasons', '121 min', '88 min', '101 min',
       '1 Season', '83 min', '100 min', '95 min', '92 min', '96 min',
       '109 min', '99 min', '75 min', '87 min', '67 min', '104 min',
       '107 min', '84 min', '103 min', '105 min', '119 min', '114 min',
       '82 min', '90 min', '130 min', '110 min', '80 min', '6 Seasons',
       '97 min', '111 min', '81 min', '49 min', '45 min', '41 min',
       '73 min', '40 min', '36 min', '39 min', '34 min', '47 min',
       '65 min', '37 min', '78 min', '102 min', '129 min', '115 min',
       '112 min', '61 min', '106 min', '76 min', '77 min', '79 min',
       '157 min', '

In [ ]:
# tomaremos los valores que contenagan la cadena de caracteres "min", "Seasons" y "Season", y la llevaremos de la columna "rating" hasta la columna "duration"
# luego los valores de columna "rating" lo llenaremos de valores np.nan 
lista_min = [r"min", r"Seasons", r"Season"]
for k in lista_min:
    lista_ind = df_concat[df_concat["rating"].str.contains(k, na = False)].index
    for i in lista_ind:
        df_concat.loc[i, "duration"] = df_concat.loc[i, "rating"]
        df_concat.loc[i, "rating"] = np.nan

In [ ]:
df_concat["rating"].unique()

array([nan, '13+', 'ALL', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+',
       'TV-PG', '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'G',
       'PG', 'NC-17', 'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES',
       'NOT_RATE', 'TV-Y7-FV', 'NOT RATED', None, 'UR'], dtype=object)

#### Seguimos con la columna cast

In [ ]:
df_concat[df_concat["cast"].isnull()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma
20,s21,TV Show,Zoboomafoo,NaN,NaN,NaN,NaN,2001,TV-Y,1 Season,Kids,Chris and Martin Kratt bring their enthusiasm ...,Amazon premier
25,s26,Movie,Zambezi: Force of Life,Dr. Rudolf Lammers,NaN,NaN,NaN,2005,ALL,50 min,Documentary,The Zambezi river is the fourth-longest river ...,Amazon premier
33,s34,Movie,Young Love (at the Sun Factory),NaN,NaN,NaN,NaN,2007,NR,36 min,Music Videos and Concerts,"Young Love celebrates the release of ""Too Youn...",Amazon premier
34,s35,Movie,Young Love (at Bowery Ballroom),NaN,NaN,NaN,NaN,2007,NR,37 min,Music Videos and Concerts,Young Love returns to Baeble's silver screen w...,Amazon premier
70,s71,Movie,World Inferno Friendship Society,Baeble Music,NaN,NaN,NaN,2007,NR,40 min,Music Videos and Concerts,The circus comes to town with this performance...,Amazon premier
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22937,s8747,Movie,Winnie,Pascale Lamche,None,"France, Netherlands, South Africa, Finland","February 26, 2018",2017,TV-14,85 min,"Documentaries, International Movies",Winnie Mandela speaks about her extraordinary ...,Netflix
22946,s8756,TV Show,Women Behind Bars,None,None,United States,"November 1, 2016",2010,TV-14,3 Seasons,"Crime TV Shows, Docuseries",This reality series recounts true stories of w...,Netflix
22947,s8757,Movie,Woodstock,Barak Goodman,None,United States,"August 13, 2019",2019,TV-MA,97 min,"Documentaries, Music & Musicals",For the 50th anniversary of the legendary Wood...,Netflix
22954,s8764,Movie,WWII: Report from the Aleutians,John Huston,None,United States,"March 31, 2017",1943,TV-PG,45 min,Documentaries,Filmmaker John Huston narrates this Oscar-nomi...,Netflix


#### Rellenamos los valores nulos con "Sin Dato"

In [ ]:
df_concat.fillna("Sin Dato", inplace = True)

#### Cambiaremos la palabra "Seasons" por "Season" en la columna "duration"

In [ ]:
df_concat["duration"] = df_concat["duration"].apply(lambda x: x.replace("Seasons","Season"))

#### Quitamos los espacios al principio y final de cada elémento en las columnas excepto en la columna "release_year" ya que esta es del tipo integer

In [ ]:
list_colum = df_concat.columns.to_list()
list_colum.remove("release_year")
for k in list_colum:
  df_concat[k].apply(lambda x: re.sub(r"^\s+", "", x))

In [ ]:
# Vemos cuantos valores nulos tenemos por cada columna y lo guardamos esto para más adelante
df_concat.replace("Sin Dato", np.nan, inplace = True)
valores_nulos_orgiginales = df_concat.isna().sum()
valores_nulos_orgiginales

show_id             0
type                0
title               0
director         8259
cast             5321
country         11499
date_added       9554
release_year        0
rating           1159
duration          187
listed_in           0
description         4
plataforma          0
dtype: int64

In [ ]:
df_concat.fillna("Sin Dato", inplace = True)

## Parte 3). Scraping a los mismos datos y web scraping. Nota importante, esto lo haremos ya que no contamos con una API ofifical para corroborar los datos sumnistrados


### Parte 3.0). Scraping a los mismos datos

#### Para hacer scraping y web scraping partiremos el dataframe en dos, haremos un dataframe para el tipo "Movie" y otro para "TV Show"

In [ ]:
df_movie = df_concat[df_concat["type"] == "Movie"]
df_serie = df_concat[df_concat["type"] == "TV Show"]

#### Primero sobre df_movie. 
##### No tendria sentido buscar en otras columnas valores repetidos ya que una pelicula puede tener el mismo director, el mismo casting, entre otras. Reduciremos solamente a buscar duplicados en la columna "titile"

In [ ]:
# Una vista de los duplicados filtrando la columna "title"
df_movie[df_movie["title"].duplicated()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma
9742,s75,Movie,Black Widow,Cate Shortland,"Scarlett Johansson, Florence Pugh, David Harbo...",United States,"October 6, 2021",2021,PG-13,135 min,"Action-Adventure, Science Fiction, Spy/Espionage",Natasha confronts her history as a spy and the...,Disney plus
10091,s424,Movie,Beauty and the Beast,Bill Condon,"Emma Watson, Dan Stevens, Luke Evans, Kevin Kl...",United States,"August 21, 2020",2017,PG,138 min,"Family, Fantasy, Musical",Belle must look beyond the Beast’s exterior to...,Disney plus
10160,s493,Movie,Tarzan,"Kevin Lima, Chris Buck","Tony Goldwyn, Minnie Driver, Glenn Close, Alex...",United States,"June 26, 2020",1999,G,93 min,"Action-Adventure, Animation, Coming of Age","Adopted by gorillas, Tarzan’s life changes for...",Disney plus
10215,s548,Movie,The Princess Bride,Rob Reiner,"Cary Elwes, Mandy Patinkin, Chris Sarandon, Ch...","United Kingdom, United States","May 1, 2020",1987,PG,99 min,"Action-Adventure, Comedy, Family",A man regales his ailing grandson with a tale ...,Disney plus
10350,s683,Movie,Wind,Edwin Chang,"Emilio Fuentes, Sonoko Konishi",United States,"December 13, 2019",2019,G,9 min,"Animation, Family, Fantasy","Trapped deep down an endless chasm, a grandmot...",Disney plus
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22913,s8723,Movie,What Lies Beneath,Robert Zemeckis,"Harrison Ford, Michelle Pfeiffer, Diana Scarwi...",United States,"January 1, 2020",2000,PG-13,130 min,"Horror Movies, Thrillers",When Claire Spencer starts hearing ghostly voi...,Netflix
22945,s8755,Movie,Wolves,Bart Freundlich,"Michael Shannon, Carla Gugino, Taylor John Smi...",United States,"March 29, 2019",2016,R,109 min,"Dramas, Independent Movies, Sports Movies",A promising high school basketball player has ...,Netflix
22948,s8758,Movie,World Trade Center,Oliver Stone,"Nicolas Cage, Michael Peña, Maggie Gyllenhaal,...",United States,"November 20, 2019",2006,PG-13,129 min,"Action & Adventure, Dramas","Working under treacherous conditions, an army ...",Netflix
22956,s8766,Movie,XX,"Jovanka Vuckovic, Annie Clark, Roxanne Benjami...","Natalie Brown, Jonathan Watton, Melanie Lynske...","Canada, United States","June 22, 2017",2017,R,81 min,Horror Movies,This four-part anthology of short horror films...,Netflix


In [ ]:
# Creamos una lista de estos duplicados
list_movie_title_dupli = df_movie[df_movie["title"].duplicated()]["title"].to_list()

##### Crearemos varias vistas sobre los algunos elementos de la lista de manera seudoaleatoria sin repetición.

In [ ]:
# Aplicamos La función random
lista = []
for i in range(5):
  dato = random.choice(list_movie_title_dupli)
  lista.append(dato)
  list_movie_title_dupli.remove(dato)

In [ ]:
df_movie[df_movie["title"] == lista[0]]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma
2044,s2045,Movie,Jeff Dunham: Controlled Chaos,"Michael Simon, Matthew McNeil",Jeff Dunham,Sin Dato,Sin Dato,2011,18+,97 min,"Arts, Entertainment, and Culture, Comedy, Spec...",Ventriloquist and comic Jeff Dunham takes to t...,Amazon premier
18199,s4009,Movie,Jeff Dunham: Controlled Chaos,"Michael Simon, Matthew McNeil",Jeff Dunham,United States,"March 15, 2019",2011,TV-MA,97 min,Stand-Up Comedy,Politically incorrect and often raunchy humor ...,Netflix


In [ ]:
df_movie[df_movie["title"] == lista[1]]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma
591,s592,Movie,The Keeping Room,Daniel Barber,"Brit Marling, Hailee Steinfeld, Sam Worthingto...",Sin Dato,Sin Dato,2015,R,95 min,"Action, Drama, Western",In this radically reimagined American Western ...,Amazon premier
11574,s457,Movie,The Keeping Room,Sin Dato,Sin Dato,Sin Dato,"July 15, 2021",2015,R,95 min,"Drama, History, Thriller",In this radically reimagined American Western ...,Hulu


In [ ]:
df_movie[df_movie["title"] == lista[2]]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma
9888,s221,Movie,Tinker Bell and the Legend of the NeverBeast,Steve Loter,"Ginnifer Goodwin, Mae Whitman, Rosario Dawson,...",United States,"May 21, 2021",2014,G,79 min,"Action-Adventure, Animation, Fantasy",Tink and her friends risk everything to rescue...,Disney plus
22785,s8595,Movie,Tinker Bell and the Legend of the NeverBeast,Steve Loter,"Ginnifer Goodwin, Mae Whitman, Rosario Dawson,...",United States,"May 17, 2015",2014,G,78 min,Children & Family Movies,When suspicious scout fairies scheme to captur...,Netflix


In [ ]:
df_movie[df_movie["title"] == lista[3]]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma
7700,s7701,Movie,The Glass House,Tom Gries,"Alan Alda, Billy Dee Williams",Sin Dato,Sin Dato,1972,13+,90 min,Drama,A young guard and a college professor convicte...,Amazon premier
11420,s303,Movie,The Glass House,Sin Dato,Sin Dato,Sin Dato,"September 1, 2021",2001,PG-13,106 min,"Drama, Horror, Thriller",Orphaned when their parents are killed in an a...,Hulu


In [ ]:
df_movie[df_movie["title"] == lista[4]]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma
7997,s7998,Movie,The Bygone,"Graham Phillips, Parker Phillips","Graham Phillips, Shawn Hatosy, Jamie McShane, ...",Sin Dato,Sin Dato,2019,18+,106 min,Western,When a young Lakota woman goes missing amongst...,Amazon premier
22424,s8234,Movie,The Bygone,"Graham Phillips, Parker Phillips","Graham Phillips, Shawn Hatosy, Jamie McShane, ...",United States,"March 23, 2020",2019,TV-MA,106 min,"Independent Movies, Thrillers",His way of life under threat due to the oil fr...,Netflix


##### Notamos algo muy peculiar, podemos decir que dos o más filas que contienen el mismo título de películas son iguales, si los valores de la columna "director" o columna "cast" o columna "release_year" o columnna "duration" de dichas filas son iguales

In [ ]:
# Aplicaremos este codigo para rellenar aquellas filas que son iguales de todo el dataframe original

df_duplicados = df_concat[df_concat["title"].duplicated()]         # Creamos una mascaras filtrando la columna "title" con títulos duplicados 
lista_duplicados = df_duplicados["title"].to_list()                # Creamos una lista con los títulos del dataframe de la linea anterior

for k in lista_duplicados:
  lista1 = df_concat[df_concat["title"] == k].index.to_list()      # Creamos una lista con los indices de la mascara al filtrar la columna "titile" con el "k-esimo" elemento de la lista anterior
  
  for j in lista1:
    lista2 = lista1.copy()
    if df_concat.loc[j, "director"] != "Sin Dato" or df_concat.loc[j, "cast"] != "Sin Dato" or df_concat.loc[j, "duration"] != "Sin Dato":
      lista2.remove(j)
      
      for t in lista2:
    
        if (df_concat.loc[j, "director"] == df_concat.loc[t, "director"] or df_concat.loc[j, "cast"] == df_concat.loc[t, "cast"] or df_concat.loc[t, "release_year"] == df_concat.loc[j, "release_year"]) or (df_concat.loc[t, "duration"] == df_concat.loc[j, "duration"]):
          
          if df_concat.loc[t, "director"] == "Sin Dato" and df_concat.loc[j, "director"] != "Sin Dato":
            df_concat.loc[t, "director"] = df_concat.loc[j, "director"]
          
          if df_concat.loc[t, "cast"] == "Sin Dato" and df_concat.loc[j, "cast"] != "Sin Dato":
            df_concat.loc[t, "cast"] = df_concat.loc[j, "cast"]
          
          if df_concat.loc[t, "country"] == "Sin Dato" and df_concat.loc[j, "country"] != "Sin Dato":
            df_concat.loc[t, "country"] = df_concat.loc[j, "country"]

          if df_concat.loc[t, "rating"] == "Sin Dato" and df_concat.loc[j, "rating"] != "Sin Dato":
            df_concat.loc[t, "rating"] = df_concat.loc[j, "rating"]
        
        else:
          continue
    else:
      continue 

#### Ahora haremos scraping sobre el tipo df_serie

In [ ]:
# Una vista de los duplicados filtrando la columna "title"
df_serie[df_serie["title"].duplicated()]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma
9757,s90,TV Show,Rolie Polie Olie,Sin Dato,"Cole Caplan, Kristen Bone, Robert Smith, Cathe...","Canada, United States, France","September 29, 2021",1998,TV-Y,5 Season,"Animation, Science Fiction",Rolie Polie Olie’s life and adventures center ...,Disney plus
11135,s18,TV Show,America's Book of Secrets,Sin Dato,Sin Dato,United States,"October 15, 2021",2012,TV-14,1 Season,"Documentaries, History",America's story can be told in many ways. Whil...,Hulu
11139,s22,TV Show,Hoarders,Sin Dato,Sin Dato,United States,"October 15, 2021",2009,TV-PG,7 Season,"Health & Wellness, Lifestyle & Culture, Reality",Each 60-minute episode of Hoarders is a fascin...,Hulu
11145,s28,TV Show,Swamp People,Sin Dato,Sin Dato,United States,"October 15, 2021",2010,TV-PG,3 Season,"Documentaries, Reality",Deep in the heart of Louisiana lies America's ...,Hulu
11166,s49,TV Show,Roots Less Traveled,Sin Dato,Sin Dato,Sin Dato,"October 10, 2021",2020,TV-G,1 Season,Reality,"Pairs of family members, often from different ...",Hulu
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22453,s8263,TV Show,The Curse of Oak Island,Sin Dato,Robert Clotworthy,United States,"September 15, 2019",2016,TV-PG,1 Season,"Docuseries, Science & Nature TV","On an ominous island off of Nova Scotia, two b...",Netflix
22502,s8312,TV Show,The Fosters,Sin Dato,"Teri Polo, Sherri Saum, Jake T. Austin, Hayden...",United States,"October 5, 2017",2017,TV-14,5 Season,TV Dramas,This offbeat drama charts the ups and downs of...,Netflix
22548,s8358,TV Show,The Inbetweeners,Sin Dato,"Simon Bird, James Buckley, Blake Harrison, Joe...",United Kingdom,"January 1, 2018",2010,TV-MA,3 Season,"British TV Shows, TV Comedies",This rib-tickling series follows four English ...,Netflix
22835,s8645,TV Show,Twin Peaks,Sin Dato,"Kyle MacLachlan, Michael Ontkean, Mädchen Amic...",United States,"July 1, 2017",1990,TV-14,2 Season,"Classic & Cult TV, Crime TV Shows, TV Dramas","""Who killed Laura Palmer?"" is the question on ...",Netflix


In [ ]:
# Creamos una lista de estos duplicados
list_serie_title_dupli = df_serie[df_serie["title"].duplicated()]["title"].to_list()

In [ ]:
# Aplicamos La función random
lista = []
for i in range(5):
  dato = random.choice(list_serie_title_dupli)
  lista.append(dato)
  list_serie_title_dupli.remove(dato)

In [ ]:
df_serie[df_serie["title"] == lista[0]]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma
12152,s1035,TV Show,Is It Wrong to Try to Pick Up Girls in a Dungeon?,Sin Dato,Sin Dato,Japan,"January 17, 2021",2015,TV-MA,3 Seasons,"Action, Adventure, Anime","In Orario, fearless adventurers band together ...",Hulu
17149,s2959,TV Show,Is It Wrong to Try to Pick Up Girls in a Dungeon?,Sin Dato,"Yoshitsugu Matsuoka, Inori Minase, Saori Onish...",Japan,"February 1, 2020",2015,TV-MA,1 Season,"Anime Series, International TV Shows","Lovable goof Bell Cranel wants an adventure, a...",Netflix


In [ ]:
df_serie[df_serie["title"] == lista[1]]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma
648,s649,TV Show,The Garfield Show,Sin Dato,Frank Welker,Sin Dato,Sin Dato,2020,ALL,4 Seasons,"Animation, Kids",A lasagne-loving cat adventures and explores t...,Amazon premier
15956,s1766,TV Show,The Garfield Show,Sin Dato,"Frank Welker, Wally Wingert, Gregg Berger","France, United States","November 1, 2020",2018,TV-Y,4 Seasons,"Kids' TV, TV Comedies","Lazy, lasagna-loving fat cat Garfield lives li...",Netflix


In [ ]:
df_serie[df_serie["title"] == lista[2]]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma
3005,s3006,TV Show,Bones,Sin Dato,"Emily Deschanel, David Boreanaz, Michaela Conlin",Sin Dato,Sin Dato,2017,NR,12 Seasons,"Drama, Suspense",The 12-episode run will celebrate the inventiv...,Amazon premier
13743,s2626,TV Show,Bones,Sin Dato,Sin Dato,United States,"September 26, 2017",2005,TV-14,12 Seasons,"Crime, Drama, Mystery",Inspired by the real-life forensic anthropolog...,Hulu


In [ ]:
df_serie[df_serie["title"] == lista[3]]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma
9760,s93,TV Show,The Simpsons,Sin Dato,"Dan Castellaneta, Julie Kavner, Nancy Cartwrig...",United States,"September 29, 2021",1989,TV-PG,32 Seasons,"Animation, Comedy","The world’s favorite nuclear family, in the aw...",Disney plus
11296,s179,TV Show,The Simpsons,Sin Dato,Sin Dato,United States,"September 27, 2021",1989,TV-PG,2 Seasons,"Adult Animation, Comedy, Sitcom",This long-running animated comedy focuses on t...,Hulu


In [ ]:
df_serie[df_serie["title"] == lista[4]]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma
2204,s2205,TV Show,Hoarders,Sin Dato,"Matt Paxton, Robin Zasio",Sin Dato,Sin Dato,2012,TV-PG,5 Seasons,"Documentary, Drama, Unscripted",Hoarders profiles adults who suffer from extre...,Amazon premier
11139,s22,TV Show,Hoarders,Sin Dato,Sin Dato,United States,"October 15, 2021",2009,TV-PG,7 Seasons,"Health & Wellness, Lifestyle & Culture, Reality",Each 60-minute episode of Hoarders is a fascin...,Hulu
16760,s2570,TV Show,Hoarders,Sin Dato,Sin Dato,Sin Dato,"May 2, 2020",2020,TV-PG,1 Season,Reality TV,This reality series enters the worlds of compu...,Netflix


In [ ]:
df_concat[df_concat["title"] == "Kill la Kill"]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma
12962,s1845,TV Show,Kill la Kill,Sin Dato,Sin Dato,Japan,"January 31, 2020",2013,TV-MA,1 Season,Anime,Ryuko Matoi is a vagrant school girl traveling...,Hulu
17928,s3738,TV Show,Kill la Kill,Sin Dato,"Ami Koshimizu, Toshihiko Seki, Ryoka Yuzuki, A...",Japan,"June 15, 2019",2014,TV-MA,1 Season,"Anime Series, International TV Shows","The ""woman with the scissor blade"" – Ryuko Mat...",Netflix


##### Podemos notar que, dos o más filas son iguales con el mismo título de serie, si los valores de la columna "director" o columna "cast" o columna "release_year" o columnna "duration" de dichas filas son iguales

In [ ]:
# Aplicaremos este código para rellenar aquellas filas que son iguales de todo el dataframe original
df_duplicados = df_concat[df_concat["title"].duplicated()]
lista_duplicados = df_duplicados["title"].to_list()

for k in lista_duplicados:
  lista1 = df_concat[df_concat["title"] == k].index.to_list()
  
  for j in lista1:
    lista2 = lista1.copy()
    if df_concat.loc[j, "director"] != "Sin Dato" or df_concat.loc[j, "cast"] != "Sin Dato" or df_concat.loc[t, "duration"] != "Sin Dato":
      lista2.remove(j)
      
      for t in lista2:
    
        if df_concat.loc[j, "director"] == df_concat.loc[t, "director"] or df_concat.loc[j, "cast"] == df_concat.loc[t, "cast"] or (df_concat.loc[t, "duration"] == df_concat.loc[j, "duration"]):
          
          if df_concat.loc[t, "director"] == "Sin Dato" and df_concat.loc[j, "director"] != "Sin Dato":
            df_concat.loc[t, "director"] = df_concat.loc[j, "director"]

          if df_concat.loc[t, "duration"] == "Sin Dato" and df_concat.loc[j, "duration"] != "Sin Dato":
            df_concat.loc[t, "duration"] = df_concat.loc[j, "duration"]
          
          if df_concat.loc[t, "cast"] == "Sin Dato" and df_concat.loc[j, "cast"] != "Sin Dato":
            df_concat.loc[t, "cast"] = df_concat.loc[j, "cast"]
          
          if df_concat.loc[t, "country"] == "Sin Dato" and df_concat.loc[j, "country"] != "Sin Dato":
            df_concat.loc[t, "country"] = df_concat.loc[j, "country"]
          
        
        else:
          continue
    else:
      continue 

#### Veamos que sucedio con el dataframe df_concat

In [ ]:
# Veamos cuantos cauntos valores nulos por columna tenemos ahora
df_concat.replace("Sin Dato", np.nan, inplace = True)
df_concat.isna().sum()

show_id             0
type                0
title               0
director         8103
cast             4984
country         11098
date_added       9554
release_year        0
rating           1088
duration          178
listed_in           0
description         4
plataforma          0
dtype: int64

In [ ]:
df_concat.fillna("Sin Dato", inplace = True)

In [ ]:
# veamos la informacion que teniamos anteriormente del df_concat
valores_nulos_orgiginales

show_id             0
type                0
title               0
director         8259
cast             5321
country         11499
date_added       9554
release_year        0
rating           1159
duration          187
listed_in           0
description         4
plataforma          0
dtype: int64

#### Notar que hubo una mejora muy pequeña

### Parte 3.1) Web scraping. Trabajaremos con 2 páginas web

#### A partir de aca trabajaremos con la librerías request y bs4. lo que haremos es hacer una petición a una página en específico con la funcion request y si esta nos da como codigo 200 empezamos a utilizar la clase Beautifulsoup para para pasear el html y poder realizar la operaciones para obtener la información que precisamos.  

In [ ]:
# Volvamos a lo anterior
df_movie = df_concat[df_concat["type"] == "Movie"]
df_serie = df_concat[df_concat["type"] == "TV Show"]

#### Web Scraping a https://www.justwatch.com/. Primero a las películas

In [ ]:
# Función para las películas
def url_movies_watch(url):
  respuesta = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.5414.74 Safari/537.36'})
              # pedimos la peticion de la url , con el headers decimos cual es el navegador web que tenemos instalado y de tal forma de noo paracer un bot
  
  dic = {}  # creamos un dicionario
  
  if respuesta.status_code == 200:                        # Si la respuesta es 200 entramos
    soup = BeautifulSoup(respuesta.text, "html.parser")   # parseamos la respuesta que nos dan con html 
    respuesta.close()

    # Tomamos Calificación, Géneros, Duración, Director
    columna1 = soup.find("div", class_ = "title-sidebar").find_all()                                # buscamos en el html conteneor "div" con la "class" y tomamos toda la informacion en una lista por ello el .find_all  
    lista_produc = ["Calificación","Géneros","Duración", 'Clasificación por edades', "Director"]

    if soup.find("div", class_ = "title-info") in columna1 or soup.find("div", class_ = "title-info") != "": 
      info_1 = soup.find("div", class_ = "title-info").find_all("div", class_ = "detail-infos")      # Bucamos el conteneor "div" con la "class" y tomamos toda la informacion que contenga el contenador que le damos con su clase
                                                                                                     # esto crea una lista cuyos elementos sera la informacion que contega el contenedor "div" y "class_ = "detail-infos"
      
      for info in info_1:
        produc = info.find("div", class_ = "detail-infos__subheading").text.strip()                  # bucamos el contenedor "div" con su clase y obtenemos solamente el texto que esta encerrado
        
        if produc in lista_produc:
          global produccion
          produccion = info.find("div", class_ = "detail-infos__value").text.strip()                # bucamos el contenedor "div" con su clase y obtenemos solamente el texto que esta encerrado
          dic[produc] = produccion
        else:
          dic[produc] = np.nan
        
    else:
      for k in lista_produc:
        dic[k] = np.nan
    
    
    # Tomamos el año
    año = soup.find("span", class_ = "text-muted").text.strip()                                    # bucamos el contenedor "span" con su clase y obtenemos solamente el texto que esta encerrado
    dic["Año"] = año

    # Tomamos los artistas de reparto
    repart = ""
    columna_2 = soup.find("div", class_ = "jw-info-box__container-content").find_all("div")        # Bucamos el conteneor "div" con la "class" y tomamos toda la informacion que contenga el contenador que le damos
                                                                                                   # Esto crea una lista con cuyos elemento son todos aquellos contenedores "div" 
    if soup.find("div", class_ = "hidden-horizontal-scrollbar title-credits") in columna_2:
      global actores
      actores = soup.find("div", class_ = "hidden-horizontal-scrollbar__items").find_all("div", class_ = "title-credits__actor") 
      
      for k in actores:                      # Notese aqui que actores es una lista donde cada elemento es la información del contenedor "div" co su calse "title-credits__actor"
        
        cast = str(k)                        # a cada elemento los volvemos una cadena
        i = cast.find("\"\"> ") + 4          # bucamos la posición de ""\ y le sumamos + 4
        j = cast.find(" </a")                # buscamos la posición espacio</a
        reparto = cast[i:j]                  # tomamos la parte de cadena que esta encerrado entre los valores i, j 
        repart += ", " + reparto        
        
      dic["Reparto"] = repart  

    else:
      dic["Reparto"] = np.nan

  else:
    pass         
  return dic


##### Creamos una lista con los títulos del dataframe df_movie, filtrando las columna "cast"

In [ ]:
lista_pelis = df_movie[df_movie["cast"] == "Sin Dato"]["title"].to_list()

##### Este codigo crea un dataframe a partir de la información que encuentre de aquellos títulos de películas que estan en la lista anterior

In [ ]:
url_prueba = "https://www.justwatch.com/en/pelicula/"
dicf = {}
df_para_conca = {}
df_para_conca = pd.DataFrame(dicf , columns= ["Título","Calificación", "Géneros","Duración", 'Clasificación por edades',"Director", "Año","Reparto"])
t = 0                                                  
dic_peli = []
for peli in lista_pelis:
  pelis = peli.lower()
  urling = str(url_prueba + pelis.replace(" ", "-"))
  
  url_peli = url_movies_watch(urling)

  if url_peli != {}:
    dic_peli =[]
    dic_title = {"Título" : peli}
    dic_title.update(url_peli)
    dic_peli.append(dic_title)
  
  else:
    continue

  df_busq2 = pd.DataFrame(dic_peli)
  df_para_conca = pd.concat([df_para_conca, df_busq2], ignore_index= True)
  print(t,peli)
  t += 1
  time.sleep(5)

In [ ]:
df_para_conca.head(5)

,Título,Calificación,Géneros,Duración,Clasificación por edades,Director,Año,Reparto
0,William the Conqueror,39% 6.1,"Historia, Documental, Drama",54min,NaN,NaN,(2013),", Dan Bronchinson, Benjamin Hubert, Antoine Le..."
1,Titus Andronicus,83% 7.8,"Drama, Terror",2h 48min,NaN,Jane Howell,(1985),", Trevor Peacock, Eileen Atkins, Anna Calder-M..."
2,Thunder Run on Baghdad,34%,,1h 57min,NaN,Jean-Pierre Krief,(2013),
3,The Road to Fame,40% 7.3,"Música, Documental, Europeas",1h 20min,NaN,Hao Wu,(2013),
4,The Night Before Christmas,39%,Animación,8min,NaN,Wilfred Jackson,(1933),", Donald Novis, Kenny Baker"


In [ ]:
# Guardamos los datos en crudo
df_para_conca.to_csv(path + "peliculas_justwatch.csv", encoding = "UTF-8", index = False)

##### Hagamos una limpieza de los datos

In [ ]:
# Creamos una copia, para trabajar comadamente
df_web1 = df_para_conca.copy()

In [ ]:
# Llenamos valores nulos con vacios
df_web1.fillna("", inplace = True)

In [ ]:
# sacamos el año que esta entre parentesis. No funciono la funcion replace
for m in range(df_web1.shape[0]):
  sentence = df_web1.loc[m, "Año"]
  s = [str(s) for s in re.findall(r'-?\d+\.?\d*', sentence)]
  df_web1.loc[m, "Año"] = s[0]
df_web1.head(5)

,Título,Calificación,Géneros,Duración,Clasificación por edades,Director,Año,Reparto
0,William the Conqueror,39% 6.1,"Historia, Documental, Drama",54min,,,2013,", Dan Bronchinson, Benjamin Hubert, Antoine Le..."
1,Titus Andronicus,83% 7.8,"Drama, Terror",2h 48min,,Jane Howell,1985,", Trevor Peacock, Eileen Atkins, Anna Calder-M..."
2,Thunder Run on Baghdad,34%,,1h 57min,,Jean-Pierre Krief,2013,
3,The Road to Fame,40% 7.3,"Música, Documental, Europeas",1h 20min,,Hao Wu,2013,
4,The Night Before Christmas,39%,Animación,8min,,Wilfred Jackson,1933,", Donald Novis, Kenny Baker"


In [ ]:
# Quitamos la "," y los espacios que estanban al comienzo de los valores de la columna "Reparto"
for k in range(df_web1.shape[0]):
    df_web1["Reparto"][k] =  df_web1["Reparto"][k][2:]

In [ ]:
# Convertimos el formato "%h  %mim" de la columna "Duración" a "% min"
for z in range(df_web1.shape[0]):
  if df_web1.loc[z, "Duración"] != "":
    sentence = df_web1.loc[z, "Duración"]
    s = [int(s) for s in re.findall(r'-?\d+\.?\d*', sentence)]
    if len(s) == 2:
      minuto = s[0] * 60 + s[1]
      df_web1.loc[z, "Duración"] = str(minuto) + " min"
      
    else:
      minuto = s[0]
      df_web1.loc[z, "Duración"] = str(minuto) + " min"
    
  else:
    continue
df_web1.head(5)   

,Título,Calificación,Géneros,Duración,Clasificación por edades,Director,Año,Reparto
0,William the Conqueror,39% 6.1,"Historia, Documental, Drama",54 min,,,2013,"Dan Bronchinson, Benjamin Hubert, Antoine Lela..."
1,Titus Andronicus,83% 7.8,"Drama, Terror",168 min,,Jane Howell,1985,"Trevor Peacock, Eileen Atkins, Anna Calder-Mar..."
2,Thunder Run on Baghdad,34%,,117 min,,Jean-Pierre Krief,2013,
3,The Road to Fame,40% 7.3,"Música, Documental, Europeas",80 min,,Hao Wu,2013,
4,The Night Before Christmas,39%,Animación,8 min,,Wilfred Jackson,1933,"Donald Novis, Kenny Baker"


In [ ]:
# Para hacer la columna "Año" del tipo integer
df_web1["Año"] = df_web1["Año"].astype(int)

In [ ]:
# Cruzamos los resultados encontrados con el dataframe "df_concat"
lis_title = df_web1["Título"].to_list()
for k in lis_title:
  index1 = df_concat[df_concat["title"] == k].index.to_list()
  index2 = df_web1[df_web1["Título"] == k].index.to_list()
  
  for l in index1:
    if df_concat.loc[l, "director"] == df_web1.loc[index2[0], "Director"] or df_concat.loc[l, "release_year"] == df_web1.loc[index2[0], "Año"] or (df_concat.loc[l, "duration"] == df_web1.loc[index2[0], "Duración"]):
      
      if df_concat.loc[l, "cast"] == "Sin Dato":
        df_concat.loc[l, "cast"] = df_web1.loc[index2[0],"Reparto"]
      
      if df_concat.loc[l, "duration"] == "Sin Dato":
        df_concat.loc[l, "duration"] = df_web1.loc[index2[0],"Duración"]
      
      if df_concat.loc[l, "director"] == "Sin Dato":
        df_concat.loc[l, "director"] = df_web1.loc[index2[0],"Director"]
      
      if df_concat.loc[l, "listed_in"] == "Sin Dato":
        df_concat.loc[l, "listed_in"] = df_web1.loc[index2[0],"Géneros"]
  

#### Ahora con las Series 

In [ ]:
# Función para las series
def url_serie_watch(url):
  respuesta = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.5414.74 Safari/537.36'})
  dic = {}
  
  if respuesta.status_code == 200:
    soup = BeautifulSoup(respuesta.text, "html.parser")
    respuesta.close()
    # Tomamos Calificación, Géneros, Duración, Director
    columna1 = soup.find("div", class_ = "title-sidebar").find_all()
    lista_produc = ["Genres","Runtime", 'Rating', "Director", "Age rating"]

    if soup.find("div", class_ = "title-info") in columna1 or soup.find("div", class_ = "title-info") != "":
      info_1 = soup.find("div", class_ = "title-info").find_all("div", class_ = "detail-infos")
      
      for info in info_1:
        produc = info.find("div", class_ = "detail-infos__subheading").text.strip()
        
        if produc in lista_produc:
          global produccion
          produccion = info.find("div", class_ = "detail-infos__value").text.strip()  
          dic[produc] = produccion
        else:
          dic[produc] = np.nan
        
    else:
      for k in lista_produc:
        dic[k] = np.nan
    
    
    # Tomamos el año
    año = soup.find("span", class_ = "text-muted").text.strip()
    dic["Year"] = año
    
    # Tomamos los artistas de reparto
    repart = ""
    columna_2 = soup.find("div", class_ = "jw-info-box__container-content").find_all("div")
    
    if soup.find("div", class_ = "hidden-horizontal-scrollbar title-credits") in columna_2:
      global actores
      actores = soup.find("div", class_ = "hidden-horizontal-scrollbar title-credits").find_all("div", class_ = "title-credits__actor")
      
      for k in actores:
        cast = str(k)
        i = cast.find("\"\"> ") + 4
        j = cast.find(" </a")
        reparto = cast[i:j]  
        repart += ", " + reparto
        
      dic["Cast"] = repart  

    else:
      dic["Cast"] = np.nan
    
    # Temporada
    plataformas = ["Amazon Prime Video", "Hulu", "Netflix", "Disney Plus"]
    temporada = soup.find("div", class_ = "price-comparison__grid__row__holder").find_all("div", class_ = "price-comparison__grid__row__element")
    
    for k in temporada:
      plate = str(k)
      i = plate.find("title=\"") + 7
      j = plate.find("\"/></")
      plataforma = str(plate[i:j])
      
      if plataforma in plataformas:
        temp = k.find("div" , class_ ="price-comparison__grid__row__price").text
        dic[plataforma] = temp
  else:
    pass         
  return dic

In [ ]:
lista_series = df_serie[df_serie["cast"] == "Sin Dato"]["title"].to_list()

In [ ]:
import time
url_prueba = "https://www.justwatch.com/us/tv-show/towies"
dicf = {}
df_para_conca = {}
df_para_conca = pd.DataFrame(dicf , columns= ["Title","Genres","Runtime", 'Rating', "Director", "Age rating", "Year", "Cast", "Amazon Prime Video", "Disney Plus","Hulu", "Netflix"])
t = 0                                                  
dic_peli = []
for peli in lista_series:
  pelis = peli.lower()
  urling = str(url_prueba + pelis.replace(" ", "-"))
  
  url_peli = url_serie_watch(urling)

  if url_peli != {}:
    dic_peli =[]
    dic_title = {"Title" : peli}
    dic_title.update(url_peli)
    dic_peli.append(dic_title)
  
  else:
    time.sleep(2)
    continue

  df_busq2 = pd.DataFrame(dic_peli)
  df_para_conca = pd.concat([df_para_conca, df_busq2], ignore_index= True)
  print(t,peli)
  t += 1
  time.sleep(3)

In [ ]:
# Guardamos los datos
df_para_conca.to_csv("series_wacth.csv", encoding = "UTF-8", sep = ",", index = False)

##### Hagamos una limpieza de los datos

In [ ]:
# Creamos una copia, para trabajar comadamente
df_web1 = df_para_conca.copy()

In [ ]:
# creamos una vista 
df_web1

,Title,Genres,Runtime,Rating,Director,Age rating,Year,Cast,Amazon Prime Video,Disney Plus,Hulu,Netflix
0,Zoboomafoo,"Animation, Comedy, Kids & Family",26min,74% 7.5 (3k),NaN,NaN,(1999),", Chris Kratt, Martin Kratt, Gord Robertson, G...",1 Season,NaN,NaN,NaN
1,WordWorld,"Animation, Kids & Family",29min,68% 6.7,NaN,TV-Y,(2007),", Veronica Taylor, Marc Thompson",1 Season HD,NaN,NaN,NaN
2,WordGirl,"Animation, Action & Adventure, Comedy, Kids & ...",19min,79% 7.1 (1k),NaN,TV-PG,(2007),", Ryan Raddatz",1 Season,NaN,NaN,NaN
3,Wonder Balls,"Comedy, Animation",12min,43% 8.8,NaN,NaN,(2015),NaN,1 Season HD,NaN,NaN,NaN
4,Wildest Places,Documentary,52min,98% 8.8,NaN,NaN,(2019),", Kevin Spink",3 Seasons HD,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1362,The Seventies,"Documentary, History",42min,89% 8.2 (1k),NaN,NaN,(2015),", Walter Cronkite, Dan Rather, Tom Hanks, Davi...",NaN,NaN,NaN,NaN
1363,Top Grier,Reality TV,22min,31% 6.5,NaN,TV-MA,(2016),NaN,NaN,NaN,NaN,NaN
1364,Towies,Reality TV,23min,35% 6.2,NaN,TV-MA,(2018),NaN,NaN,NaN,NaN,NaN
1365,Women Behind Bars,Documentary,42min,63% 6.7,NaN,TV-14,(2008),NaN,NaN,NaN,NaN,NaN


In [ ]:
# Veamos cuantos valores nulos hay por columna
df_web1.isna().sum()

Title                    0
Genres                  28
Runtime                  6
Rating                   0
Director              1230
Age rating             270
Year                     0
Cast                   323
Amazon Prime Video    1196
Disney Plus           1315
Hulu                   767
Netflix               1238
dtype: int64

In [ ]:
# Llenamos valores nulos con vacios
df_web1.fillna("", inplace = True)

In [ ]:
# Sacamos el año que esta entre parentesis.
for m in range(df_web1.shape[0]):
  sentence = df_web1.loc[m, "Year"]
  s = [str(s) for s in re.findall(r'-?\d+\.?\d*', sentence)]
  df_web1.loc[m, "Year"] = s[0]
df_web1.head(5)

,Title,Genres,Runtime,Rating,Director,Age rating,Year,Cast,Amazon Prime Video,Disney Plus,Hulu,Netflix
0,Zoboomafoo,"Animation, Comedy, Kids & Family",26min,74% 7.5 (3k),,,1999,", Chris Kratt, Martin Kratt, Gord Robertson, G...",1 Season,,,
1,WordWorld,"Animation, Kids & Family",29min,68% 6.7,,TV-Y,2007,", Veronica Taylor, Marc Thompson",1 Season HD,,,
2,WordGirl,"Animation, Action & Adventure, Comedy, Kids & ...",19min,79% 7.1 (1k),,TV-PG,2007,", Ryan Raddatz",1 Season,,,
3,Wonder Balls,"Comedy, Animation",12min,43% 8.8,,,2015,,1 Season HD,,,
4,Wildest Places,Documentary,52min,98% 8.8,,,2019,", Kevin Spink",3 Seasons HD,,,


In [ ]:
# Quitamos la "," y los espacios que estanban al comienzo de los valores de la columna "Reparto"
for k in range(df_web1.shape[0]):
    df_web1["Cast"][k] =  df_web1["Cast"][k][2:]

In [ ]:
# Convertimos el formato "%h  %mim" de la columna "Duración" a "% min"
for z in range(df_web1.shape[0]):
  if df_web1.loc[z, "Runtime"] != "":
    sentence = df_web1.loc[z, "Runtime"]
    s = [int(s) for s in re.findall(r'-?\d+\.?\d*', sentence)]
    if len(s) == 2:
      minuto = s[0] * 60 + s[1]
      df_web1.loc[z, "Runtime"] = str(minuto) + " min"
      
    else:
      minuto = s[0]
      df_web1.loc[z, "Runtime"] = str(minuto) + " min"
    
  else:
    continue
df_web1.head(5)   

,Title,Genres,Runtime,Rating,Director,Age rating,Year,Cast,Amazon Prime Video,Disney Plus,Hulu,Netflix
0,Zoboomafoo,"Animation, Comedy, Kids & Family",26 min,74% 7.5 (3k),,,1999,"Chris Kratt, Martin Kratt, Gord Robertson, Gen...",1 Season,,,
1,WordWorld,"Animation, Kids & Family",29 min,68% 6.7,,TV-Y,2007,"Veronica Taylor, Marc Thompson",1 Season HD,,,
2,WordGirl,"Animation, Action & Adventure, Comedy, Kids & ...",19 min,79% 7.1 (1k),,TV-PG,2007,Ryan Raddatz,1 Season,,,
3,Wonder Balls,"Comedy, Animation",12 min,43% 8.8,,,2015,,1 Season HD,,,
4,Wildest Places,Documentary,52 min,98% 8.8,,,2019,Kevin Spink,3 Seasons HD,,,


In [ ]:
# Hacemos tos los elementos de la columna "Year" a enteros
df_web1["Year"] = df_web1["Year"].astype(int)

In [ ]:
# Quitamos Espacios en las 4 ultimas columnas
plataformas = ["Amazon Prime Video", "Hulu", "Netflix", "Disney Plus"]
for k in plataformas:
  df_web1[k] = df_web1[k].apply(lambda x: x.strip())

In [ ]:
# Reeplaczamos "Season HD", "Seasons HD" y "Seasons" por "Season".
for k in plataformas:
  df_web1[k] = df_web1[k].apply(lambda x: x.replace("Season HD", "Seasons"))
  df_web1[k] = df_web1[k].apply(lambda x: x.replace("Seasons HD", "Seasons"))
  df_web1[k] = df_web1[k].apply(lambda x: x.replace("Seasons", "Season"))

In [ ]:
#Cambiaremos los nombres de algunas columnas para luego hacer el cruzamiento de información
df_web1.rename(columns={"Amazon Prime Video": "Amazon premier", "Disney Plus": "Disney plus"}, inplace=True)
df_web1

,Title,Genres,Runtime,Rating,Director,Age rating,Year,Cast,Amazon premier,Disney plus,Hulu,Netflix
0,Zoboomafoo,"Animation, Comedy, Kids & Family",26 min,74% 7.5 (3k),,,1999,"Chris Kratt, Martin Kratt, Gord Robertson, Gen...",1 Season,,,
1,WordWorld,"Animation, Kids & Family",29 min,68% 6.7,,TV-Y,2007,"Veronica Taylor, Marc Thompson",1 Season,,,
2,WordGirl,"Animation, Action & Adventure, Comedy, Kids & ...",19 min,79% 7.1 (1k),,TV-PG,2007,Ryan Raddatz,1 Season,,,
3,Wonder Balls,"Comedy, Animation",12 min,43% 8.8,,,2015,,1 Season,,,
4,Wildest Places,Documentary,52 min,98% 8.8,,,2019,Kevin Spink,3 Season,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
1362,The Seventies,"Documentary, History",42 min,89% 8.2 (1k),,,2015,"Walter Cronkite, Dan Rather, Tom Hanks, David ...",,,,
1363,Top Grier,Reality TV,22 min,31% 6.5,,TV-MA,2016,,,,,
1364,Towies,Reality TV,23 min,35% 6.2,,TV-MA,2018,,,,,
1365,Women Behind Bars,Documentary,42 min,63% 6.7,,TV-14,2008,,,,,


In [ ]:
# Comapremos los resultados encontrados con el dataframe "df_concat"
lis_title = df_web1["Title"].to_list()
for k in lis_title:
  index1 = df_concat[df_concat["title"] == k].index.to_list()
  index2 = df_web1[df_web1["Title"] == k].index.to_list()
  
  for l in index1:
    plataforma = df_concat.loc[l, "plataforma"]

    if df_concat.loc[l, "director"] == df_web1.loc[index2[0], "Director"] or (df_web1.loc[index2[0], plataforma] != "") or df_concat.loc[l, "release_year"] == df_web1.loc[index2[0], "Year"]:
      
      if df_concat.loc[l, "cast"] == "Sin Dato" and df_web1.loc[index2[0],"Cast"] != "":
        df_concat.loc[l, "cast"] = df_web1.loc[index2[0],"Cast"]
      
      if df_concat.loc[l, "duration"] == "Sin Dato" and (df_web1.loc[index2[0], plataforma] != ""):
          df_concat.loc[l, "duration"] = df_web1.loc[index2[0], plataforma]

      if df_concat.loc[l, "director"] == "Sin Dato" and df_web1.loc[index2[0],"Director"] != "":
        df_concat.loc[l, "director"] = df_web1.loc[index2[0],"Director"]
      
      if df_concat.loc[l, "listed_in"] == "Sin Dato" and df_web1.loc[index2[0],"Genres"] != "":
        df_concat.loc[l, "listed_in"] = df_web1.loc[index2[0],"Genres"]

      if df_concat.loc[l, "rating"] == "Sin Dato" and  df_web1.loc[index2[0],"Age rating"] != "":
        df_concat.loc[l, "rating"] = df_web1.loc[index2[0],"Age rating"]        
  

In [ ]:
# Veamos si hubo alguna mejora
df_concat.replace("Sin Dato", np.nan, inplace = True)
df_concat.isna().sum()

show_id             0
type                0
title               0
director         7825
cast             3733
country         11098
date_added       9554
release_year        0
rating           1056
duration          178
listed_in           0
description         4
plataforma          0
dtype: int64

In [ ]:
df_concat.fillna("Sin Dato", inplace = True)

In [ ]:
valores_nulos_orgiginales

show_id             0
type                0
title               0
director         8259
cast             5321
country         11499
date_added       9554
release_year        0
rating           1159
duration          187
listed_in           0
description         4
plataforma          0
dtype: int64

#### Web scraping a "https://www.rottentomatoes.com/m/". Primero Películas

In [ ]:
# Tomaremos aquellos valores nulos que quedaron despues de la actualización del df_concat original
df_movie = df_concat[df_concat["type"] == "Movie"]
df_serie = df_concat[df_concat["type"] == "TV Show"]

In [ ]:
# Función para las películas
def url_rotten_movies(url):
  respuesta = requests.get(url)
  dic = {}
  
  if respuesta.status_code == 200:
    soup = BeautifulSoup(respuesta.text, "html.parser")
    respuesta.close()
    
    columna = soup.find("div", id = "mainColumn").find_all()
    actores = ""
    
    if soup.find("div", class_ = "castSection") in columna:
      castin = soup.find("div", class_ = "castSection").find_all("div", class_ = "media-body")
    
      lista_produc = ["Director","Screenwriter","Producer", "Executive Producer", "Cinematographer", "Film Editing"]

      for cast in castin:
        produc = cast.find("span", class_ = "characters subtle smaller").text.strip()
        if produc not in lista_produc:
          if cast.find("a", class_ = "unstyled articleLink") in cast:
            actor = cast.find("a", class_ = "unstyled articleLink").text.strip()
            actores += ", " + actor
          else:
            pass
        else:
          break
    else:
      actores = np.nan
      dic["cast"] = actores
      pass
    
    dic["cast"] = actores


    lista_partes_info = ["Rating", "Genre", "Original Language", "Director", "Producer",
                         "Writer", "Release Date (Theaters)", "Release Date (Streaming)",
                         "Box Office (Gross USA)", "Runtime", "Distributor", "Sound Mix", 
                         "Aspect Ratio"]
      
    info = soup.find("ul", class_ = "content-meta info").find_all("li", class_ = "meta-row clearfix")
    j, k = 0,0
    while j <= len(lista_partes_info) - 1:
      if k < len(info):
        llave = info[k].find("div", class_ = "meta-label subtle").text.strip("\n").replace(":","")
        if lista_partes_info[j] == llave:
          valor = info[k].find("div", class_ = "meta-value").get_text(strip = True, separator = " ")
          dic[lista_partes_info[j]] = valor
          k += 1
        else:
          dic[lista_partes_info[j]] = np.nan
      else:
        dic[lista_partes_info[j]] = np.nan
      j += 1
  else:
    pass 
  return dic

In [ ]:
lista_pelis = df_movie[df_movie["cast"] == "Sin Dato"]["title"].to_list()

In [ ]:
url_prueba = "https://www.rottentomatoes.com/m/" # pagina semilla
dicf = {}
df_para_conca = {}
df_para_conca = pd.DataFrame(dicf , columns= ["title","cast","Rating", "Genre", "Original Language", "Director", "Producer",
                                                  "Writer", "Release Date (Theaters)", "Release Date (Streaming)",
                                                  "Box Office (Gross USA)", "Runtime", "Distributor", "Sound Mix", 
                                                  "Aspect Ratio"])

dic_peli = []
t = 0

for peli in lista_pelis:
  peli.lower()
  try:
    peli_info = url_rotten_movies(url_prueba + peli.replace(" ", "_"))
  except:
    continue
  
  if peli_info != {}:
    dic_peli = []
    dic_title = {"title": peli}
    dic_title.update(peli_info)
    dic_peli.append(dic_title)
    
  else:
    print(t, peli)
    t += 1
    time.sleep(2)
    continue

  df_busqueda1 = pd.DataFrame(dic_peli)                                        # crea un dataframe a partir de una lista que contiene un dicionario como valor 
  df_para_conca = pd.concat([df_para_conca, df_busqueda1], ignore_index = True) # concatenamos cada dataframe creado anteriormente
  print(t, peli)
  t += 1
  time.sleep(3)

In [ ]:
#guardamos el dataframe como un archivo csv
df_para_conca.to_csv(path + "pelis_rotten.csv",encoding="UTF-8" , index = False, sep = ",")

In [ ]:
# Creamos una copia
df_web2 = df_para_conca.copy()

In [ ]:
df_web2 = pd.read_csv("pelis_rotten.csv", sep = ",")

In [ ]:
# Una vista del dataframe
df_web2.head(5)

,title,cast,Rating,Genre,Original Language,Director,Producer,Writer,Release Date (Theaters),Release Date (Streaming),Box Office (Gross USA),Runtime,Distributor,Sound Mix,Aspect Ratio
0,William the Conqueror,", Tiésay Deshayes, Jean-Damien Détouillon, Dan...",NaN,"Adventure, \n \n ...",French (France),Fabien Drugeon,Jonathan Perrut,Fabien Drugeon,NaN,"Mar 30, 2020",NaN,1h 30m,NaN,NaN,NaN
1,Wild Colombia Revealed,", Julio Sánchez Cristo",NaN,Documentary,Spanish,Mike Slee,NaN,Mike Slee,NaN,"May 23, 2017",NaN,1h 30m,NaN,NaN,NaN
2,Undocument,", Julia Krynke, Nabil Elouahabi, Molly Jones, ...",NaN,Drama,English (United Kingdom),"Amin Bakhshian , Kyla Simone Bruce","Amin Bakhshian , Kyla Simone Bruce","Amin Bakhshian , Kyla Simone Bruce",NaN,"Jun 28, 2020",NaN,1h 33m,NaN,NaN,NaN
3,Tour Balas Perdidas en Madrid,", Morat, Aitana, Cali Y El Dandee, Antonio José",NaN,"Documentary, \n \n ...",Spanish,Carlos Sanchez-Llibre,NaN,NaN,NaN,NaN,NaN,1h 56m,NaN,NaN,NaN
4,The Wild Horse Redemption,NaN,NaN,Documentary,English,John Zaritsky,NaN,"David Cruise , Alison Griffiths",NaN,"Jun 21, 2018",NaN,1h 31m,NaN,NaN,NaN


In [ ]:
# Cambiamos los valores nulos por ""
df_web2.fillna("", inplace = True)

In [ ]:
# Limpieza de la columna "cast"
for k in range(df_web2.shape[0]):
    df_web2["cast"][k] =  df_web2["cast"][k][2:]

In [ ]:
df_web2.head(5)

,title,cast,Rating,Genre,Original Language,Director,Producer,Writer,Release Date (Theaters),Release Date (Streaming),Box Office (Gross USA),Runtime,Distributor,Sound Mix,Aspect Ratio
0,William the Conqueror,"Tiésay Deshayes, Jean-Damien Détouillon, Dan B...",,"Adventure, \n \n ...",French (France),Fabien Drugeon,Jonathan Perrut,Fabien Drugeon,,"Mar 30, 2020",,1h 30m,,,
1,Wild Colombia Revealed,Julio Sánchez Cristo,,Documentary,Spanish,Mike Slee,,Mike Slee,,"May 23, 2017",,1h 30m,,,
2,Undocument,"Julia Krynke, Nabil Elouahabi, Molly Jones, Um...",,Drama,English (United Kingdom),"Amin Bakhshian , Kyla Simone Bruce","Amin Bakhshian , Kyla Simone Bruce","Amin Bakhshian , Kyla Simone Bruce",,"Jun 28, 2020",,1h 33m,,,
3,Tour Balas Perdidas en Madrid,"Morat, Aitana, Cali Y El Dandee, Antonio José",,"Documentary, \n \n ...",Spanish,Carlos Sanchez-Llibre,,,,,,1h 56m,,,
4,The Wild Horse Redemption,,,Documentary,English,John Zaritsky,,"David Cruise , Alison Griffiths",,"Jun 21, 2018",,1h 31m,,,


In [ ]:
# Limpieza de la columna "Genre"
df_web2["Genre"] = df_web2["Genre"].replace('\n', '')
df_web2["Genre"] = df_web2["Genre"].replace(' ', '')
for i in range(df_web2.shape[0]):
  df_web2.loc[i, "Genre"] = re.sub(r'\n', '', df_web2.loc[i, "Genre"])
  re.sub(r' ', '', df_web2.loc[i, "Genre"])

In [ ]:
# Obtenomos solamente el año de la columna "Release Date (Streaming)"
for k in range(df_web2.shape[0]):
    date = df_web2.loc[k, "Release Date (Streaming)"].split(" ")
    if date != [""] or len(date) == 3:
      año = date[2]
      df_web2.loc[k, "Release Date (Streaming)"] = año
    else:
      continue

In [ ]:
# Convertimos el formato "%h  %mim" de la columna "Runtime" a "% min"
for z in range(df_web2.shape[0]):
  if df_web2.loc[z, "Runtime"] != "":
    sentence = df_web2.loc[z, "Runtime"]
    s = [int(s) for s in re.findall(r'-?\d+\.?\d*', sentence)]
    if len(s) == 2:
      minuto = s[0] * 60 + s[1]
      df_web2.loc[z, "Runtime"] = str(minuto) + " min"
      
    else:
      minuto = s[0]
      df_web2.loc[z, "Runtime"] = str(minuto) + " min"
      
  else:
    continue

In [ ]:
df_web2["Release Date (Streaming)"].replace("", 0, inplace = True)
df_web2["Release Date (Streaming)"] = df_web2["Release Date (Streaming)"].astype(int)
df_web2.head(5)

,title,cast,Rating,Genre,Original Language,Director,Producer,Writer,Release Date (Theaters),Release Date (Streaming),Box Office (Gross USA),Runtime,Distributor,Sound Mix,Aspect Ratio
0,William the Conqueror,"Tiésay Deshayes, Jean-Damien Détouillon, Dan B...",,"Adventure, ...",French (France),Fabien Drugeon,Jonathan Perrut,Fabien Drugeon,,2020,,90 min,,,
1,Wild Colombia Revealed,Julio Sánchez Cristo,,Documentary,Spanish,Mike Slee,,Mike Slee,,2017,,90 min,,,
2,Undocument,"Julia Krynke, Nabil Elouahabi, Molly Jones, Um...",,Drama,English (United Kingdom),"Amin Bakhshian , Kyla Simone Bruce","Amin Bakhshian , Kyla Simone Bruce","Amin Bakhshian , Kyla Simone Bruce",,2020,,93 min,,,
3,Tour Balas Perdidas en Madrid,"Morat, Aitana, Cali Y El Dandee, Antonio José",,"Documentary, ...",Spanish,Carlos Sanchez-Llibre,,,,0,,116 min,,,
4,The Wild Horse Redemption,,,Documentary,English,John Zaritsky,,"David Cruise , Alison Griffiths",,2018,,91 min,,,


In [ ]:
# Cruzemos los resultados encontrados con el dataframe "df_concat"
lis_title = df_web2["title"].to_list()
for k in lis_title:
  index1 = df_concat[df_concat["title"] == k].index.to_list()
  index2 = df_web2[df_web2["title"] == k].index.to_list()
  
  for l in index1:
    if df_concat.loc[l, "director"] == df_web2.loc[index2[0], "Director"] or df_concat.loc[l, "release_year"] == df_web2.loc[index2[0], "Release Date (Streaming)"] or ((df_concat.loc[l, "release_year"] == df_web2.loc[index2[0], "Release Date (Streaming)"] - 1 or df_concat.loc[l, "release_year"] - 1 == df_web2.loc[index2[0], "Release Date (Streaming)"]) and df_concat.loc[l, "duration"] == df_web2.loc[index2[0], "Runtime"]):
      
      if df_concat.loc[l, "cast"] == "Sin Dato" and df_web2.loc[index2[0],"cast"] != "":
        df_concat.loc[l, "cast"] = df_web2.loc[index2[0],"cast"]
      
      if df_concat.loc[l, "duration"] == "Sin Dato" and df_web2.loc[index2[0],"Runtime"] != "":
        df_concat.loc[l, "duration"] = df_web2.loc[index2[0],"Runtime"]
      
      if df_concat.loc[l, "director"] == "Sin Dato" and df_web2.loc[index2[0],"Director"] != "":
        df_concat.loc[l, "director"] = df_web2.loc[index2[0],"Director"]
      
      if df_concat.loc[l, "duration"] == "Sin Dato" and df_web2.loc[index2[0],"Genre"] != "":
        df_concat.loc[l, "duration"] = df_web2.loc[index2[0],"Genre"]

#### Ahora Para las Series

In [ ]:
# Función para las series
def url_rotten_series(url):
  respuesta = requests.get(url)
  dic = {}
  
  if respuesta.status_code == 200:
    soup = BeautifulSoup(respuesta.text, "html.parser")
    respuesta.close()
    
    columna = soup.find("div", id = "mainColumn").find_all()
    actores = ""
    
    if soup.find("div", class_ = "castSection") in columna:
      castin = soup.find("div", class_ = "castSection").find_all("div", class_ = "media-body")
    
      lista_produc = ["Director","Screenwriter","Producer", "Executive Producer", "Cinematographer", "Film Editing"]

      for cast in castin:
        produc = cast.find("span", class_ = "characters subtle smaller").text.strip()
        if produc not in lista_produc:
          if cast.find("a", class_ = "unstyled articleLink") in cast:
            actor = cast.find("a", class_ = "unstyled articleLink").text.strip()
            actores += ", " + actor
          else:
            pass
        else:
          break
    else:
      actores = np.nan
      dic["cast"] = actores
      pass
    
    dic["cast"] = actores

    info = soup.find("section", id = "detail_panel").find_all("tr")
  
    for k in info:
      producer = k.find("td", class_ = "fgm").text.strip()
  
      if producer != "Executive producers:":
        lupa = str(k)
        i = lupa.find("/td>") 
        lupa = str(lupa[i:])
        p = lupa.find("\">") + 2
        j = lupa.find("</t")
        texto = lupa[p:j]
        dic[producer] = texto
      
      else:
        productores = ""
        lupa = str(k)
        i = lupa.find("<a") 
        lupa = str(lupa[i:]).split(",")
        for l in lupa:
          l_split = str(l)
          p = l_split.find("\">") + 2
          j = l_split.find("</a>")
          texto = str(l[p:j])
          productores += texto + ", "
        
        dic["Executive producers"] = productores
  else:
    pass 
  return dic

In [ ]:
url_prueba = "https://www.rottentomatoes.com/tv/the_legend_of_vox_machina"
url_rotten_series(url_prueba)

{'cast': ", Laura Bailey, Taliesin Jaffe, Ashley Johnson, Liam O'Brien, Marisha Ray, Sam Riegel, Travis Willingham, Matthew Mercer",
 'TV Network:': 'Prime Video',
 'Premiere Date:': 'Jan 28, 2022',
 'Genre:': 'Fantasy',
 'Executive producers': 'Brandon Auman, Chris Prynoski, '}

In [ ]:
lista_pelis = df_serie[df_serie["cast"] == "Sin Dato"]["title"].to_list()

In [ ]:
url_prueba = "https://www.rottentomatoes.com/tv/" # pagina semilla
dicf = {}
df_para_conca = {}
df_para_conca = pd.DataFrame(dicf , columns= ["title","cast","Rating", "Genre", "Original Language", "Director", "Producer",
                                                  "Writer", "Release Date (Theaters)", "Release Date (Streaming)",
                                                  "Box Office (Gross USA)", "Runtime", "Distributor", "Sound Mix", 
                                                  "Aspect Ratio"])

dic_peli = []
t = 0

for peli in lista_pelis:
  peli.lower()
  try:
    peli_info = url_rotten_movies(url_prueba + peli.replace(" ", "_"))
  except:
    continue
  
  if peli_info != {}:
    dic_peli = []
    dic_title = {"title": peli}
    dic_title.update(peli_info)
    dic_peli.append(dic_title)
    
  else:
    time.sleep(2)
    continue

  df_busqueda1 = pd.DataFrame(dic_peli)                                        # crea un dataframe a partir de una lista que contiene un dicionario como valor 
  df_para_conca = pd.concat([df_para_conca, df_busqueda1], ignore_index = True) # concatenamos cada dataframe creado anteriormente
  print(t, peli)
  t += 1
  time.sleep(3)

##### No pudo encontar ninguna información sobre las series que le suministramos

In [ ]:
# Veamos si hubo alguna mejora
df_concat.replace("Sin Dato", np.nan, inplace = True)
df_concat.isna().sum()

show_id             0
type                0
title               0
director         7825
cast             3733
country         11098
date_added       9554
release_year        0
rating           1056
duration          178
listed_in           0
description         4
plataforma          0
dtype: int64

In [ ]:
valores_nulos_orgiginales

show_id             0
type                0
title               0
director         8259
cast             5321
country         11499
date_added       9554
release_year        0
rating           1159
duration          187
listed_in           0
description         4
plataforma          0
dtype: int64

In [ ]:
df_concat.fillna("Sin Dato", inplace = True)

## Parte 4). Normalizar el dataframe "df_concat"

In [ ]:
#vemos la informacion del dataframe
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       22998 non-null  object
 1   type          22998 non-null  object
 2   title         22998 non-null  object
 3   director      22998 non-null  object
 4   cast          22998 non-null  object
 5   country       22998 non-null  object
 6   date_added    22998 non-null  object
 7   release_year  22998 non-null  int64 
 8   rating        22998 non-null  object
 9   duration      22998 non-null  object
 10  listed_in     22998 non-null  object
 11  description   22998 non-null  object
 12  plataforma    22998 non-null  object
dtypes: int64(1), object(12)
memory usage: 2.3+ MB


In [ ]:
# Creamos una vista
df_concat

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,Sin Dato,113 min,"Comedy, Drama",A small fishing village must procure a local d...,Amazon premier
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,Amazon premier
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,Sin Dato,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,Amazon premier
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,Sin Dato,69 min,Documentary,"Pink breaks the mold once again, bringing her ...",Amazon premier
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,Sin Dato,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,Amazon premier
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22993,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a...",Netflix
22994,s8804,TV Show,Zombie Dumb,Sin Dato,Sin Dato,Sin Dato,"July 1, 2019",2018,TV-Y7,2 Season,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g...",Netflix
22995,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...,Netflix
22996,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero...",Netflix


In [ ]:
# Crearemos nuevos valores para la columna "show_id" a partir de la indexación
for i in range(df_concat.shape[0]):
    df_concat.loc[i, "show_id"] = "s" + str(i + 1)

In [ ]:
# Creamos la columna "min", con sus valores correspondientes
df_concat["min"] = ""
for k in range(df_concat.shape[0]):
  sprit = df_concat.loc[k, "duration"].split(" ")
  if sprit[1] == "min":
    df_concat.loc[k, "min"] = sprit[0]
  else:
    df_concat.loc[k, "min"] = 0

In [ ]:
# Creamos la columna "Season", con sus valores correspondientes
df_concat["Season"] = ""
for k in range(df_concat.shape[0]):
  sprit = df_concat.loc[k, "duration"].split(" ")
  if sprit[1] == "Season":
    df_concat.loc[k, "Season"] = sprit[0]
  else:
    df_concat.loc[k, "Season"] = 0

In [ ]:
# Creamos una vista
df_concat

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataforma,min,Season
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,Sin Dato,113 min,"Comedy, Drama",A small fishing village must procure a local d...,Amazon premier,113,0
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,Amazon premier,110,0
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,Sin Dato,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,Amazon premier,74,0
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,Sin Dato,69 min,Documentary,"Pink breaks the mold once again, bringing her ...",Amazon premier,69,0
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,Sin Dato,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,Amazon premier,45,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22993,s22994,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a...",Netflix,158,0
22994,s22995,TV Show,Zombie Dumb,Sin Dato,Sin Dato,Sin Dato,"July 1, 2019",2018,TV-Y7,2 Season,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g...",Netflix,0,2
22995,s22996,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...,Netflix,88,0
22996,s22997,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero...",Netflix,88,0


### Cambiado de posición la columna "min" y "Seasons"

In [ ]:
k_m = df_concat.pop("min")
k_s = df_concat.pop("Season")
df_concat.insert(9, "min", k_m)
df_concat.insert(10, "season", k_s)

#### Eliminamos la columna "duration"

In [ ]:
df_concat.drop("duration", axis = 1, inplace = True)

### Cambiamos el nombre de la columna "release_year" por "year"

In [ ]:
df_concat.rename(columns={"release_year": "year"}, inplace=True)

In [ ]:
df_concat

,show_id,type,title,director,cast,country,date_added,year,rating,min,season,listed_in,description,plataforma
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,Sin Dato,113,0,"Comedy, Drama",A small fishing village must procure a local d...,Amazon premier
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110,0,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,Amazon premier
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,Sin Dato,74,0,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,Amazon premier
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,Sin Dato,69,0,Documentary,"Pink breaks the mold once again, bringing her ...",Amazon premier
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,Sin Dato,45,0,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,Amazon premier
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22993,s22994,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158,0,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a...",Netflix
22994,s22995,TV Show,Zombie Dumb,Sin Dato,Sin Dato,Sin Dato,"July 1, 2019",2018,TV-Y7,0,2,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g...",Netflix
22995,s22996,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88,0,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...,Netflix
22996,s22997,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88,0,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero...",Netflix


### Guardamos el dataframe como un archivo CSV

In [ ]:
df_concat.to_csv("api_movie_tvshow.csv", sep = ",", encoding = "UTF-8", index = False)